In [1]:
import pandas as pd
from Levenshtein import matching_blocks, editops, ratio
import difflib

In [2]:
!pip install Levenshtein

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip


# Read data

In [3]:
brands_1_path = "data/brands_list_1_short.csv"
brands_2_path = "data/brands_list_2_short.csv"

brands_1 = pd.read_csv(brands_1_path, delimiter=';', header=None)[0]
brands_2 = pd.read_csv(brands_2_path, delimiter=';', header=None)[0]

print(brands_1)
print(brands_2)

0               2 EMME CLIMA
1              3A COMPOSITES
2                    3-EN-UN
3                         3M
4                        A2C
                ...         
1043               KRONOSPAN
1044    KRONOSPAN LUXEMBOURG
1045                     KSB
1046           KTL CERAMICAS
1047                     KWC
Name: 0, Length: 1048, dtype: object
0      3M BRICOLAGE ET BATIMENT
1                         A.T.E
2                  A2C GRANULAT
3                 ABIES AUSTRIA
4        ABM AIX BOIS MATERIAUX
                 ...           
668          KRONIMUS MAIZIERES
669     KRONOSPAN LUXEMBOURG SA
670               KRONOSPAN SAS
671                    KS TOOLS
672        KYOCERA SENCO FRANCE
Name: 0, Length: 673, dtype: object


# Similar words
Find exact word match between a brand and a series of brands

In [4]:
def similar_words(compared_brand, series_of_brand):
    """
    Compare one word with a series and find number of similar words. 

    Parameters:
    - compared_brand (string): Compared brand.
    - series_of_brand (pd.Series): Series of other brands to compare to. 
    
    Returns: 
    A sorted dataframe where there is at least one similar word.
    """
    intersections = pd.DataFrame(columns=['compared_brand', 'other_brand', 'similar_words'])
    
    for i in range(len(series_of_brand)):
        other_brand = series_of_brand[i]
        intersections.loc[i, 'compared_brand'] = compared_brand
        intersections.loc[i, 'other_brand'] = other_brand
        similar_words = len(set(compared_brand.split()).intersection(set(other_brand.split())))
        intersections.loc[i, 'similar_words'] = similar_words
    
    return intersections[intersections.similar_words > 0].sort_values(by='similar_words', ascending=False)

In [5]:
similar_words('BETON', brands_2)

,compared_brand,other_brand,similar_words
13,BETON,ADG BETON INDUSTRIEL SA,1
125,BETON,BETON CONTROLE GARD ARDEC,1
126,BETON,BETON DE LA HAUTE-SEINE,1
127,BETON,BETON VICAT,1
206,BETON,CBS BETON,1
324,BETON,DIJON BETON,1
392,BETON,EQIOM GROUPE CRH - BETON,1
462,BETON,FINGER BETON KUHARDT,1
521,BETON,GISONE BETON VIBRE,1


In [6]:
similar_words('BETONS', brands_2)

,compared_brand,other_brand,similar_words
128,BETONS,BETONS MATERIAUX CONTROLE,1
129,BETONS,BETONS OCCITANS,1
211,BETONS,CEMEX BETONS SUD EST,1
212,BETONS,CEMEX BETONS SUD OUEST,1


# Levenshtein Ratio
https://rapidfuzz.github.io/Levenshtein/levenshtein.html#Levenshtein.ratio

Calculates a normalized indel similarity in the range [0, 1]. The indel distance calculates the minimum number of insertions and deletions required to change one sequence into the other.

In [7]:
def calculate_levenshtein_ratio(compared_brand, series_of_brand):
    """
    Calculates ratio between a word and a series. 

    Parameters:
    - compared_brand (string): Compared brand.
    - series_of_brand (pd.Series): Series of other brands to compare to. 
    
    Returns: 
    A sorted dataframe where ratio is superior to 0.5.
    """
    res_ratio = pd.Series(index=range(len(series_of_brand)), name='ratio')
    for i in range(len(series_of_brand)):
        res_ratio[i] = ratio(compared_brand, series_of_brand[i])
    
    res_merged = pd.merge(series_of_brand, res_ratio, left_index=True, right_index=True)
    sorted_ratios = res_merged.sort_values(by='ratio', ascending=False)

    return sorted_ratios

In [8]:
calculate_levenshtein_ratio('MONOCIBEC', brands_2)

/var/folders/m_/2w2qlxsn13s9qyf11rtgzkkr0000gn/T/ipykernel_81706/3197432269.py:14: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  res_ratio = pd.Series(index=range(len(series_of_brand)), name='ratio')


,0,ratio
459,FINCIBEC (MONOCIBEC),0.620690
330,DISMO FRANCE,0.476190
267,CONCILIO,0.470588
158,BONNEFOY BBCI,0.454545
387,EPDM PROVENCE,0.454545
...,...,...
86,AUDY SAS,0.000000
356,DYKA SAS,0.000000
649,JTS,0.000000
516,GFD,0.000000


In [9]:
calculate_levenshtein_ratio('BETON', brands_2)

/var/folders/m_/2w2qlxsn13s9qyf11rtgzkkr0000gn/T/ipykernel_81706/3197432269.py:14: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  res_ratio = pd.Series(index=range(len(series_of_brand)), name='ratio')


,0,ratio
121,BERGON,0.727273
206,CBS BETON,0.714286
127,BETON VICAT,0.625000
324,DIJON BETON,0.625000
632,ITRON,0.600000
...,...,...
284,CRP,0.000000
439,FDI - SAS,0.000000
63,ARCM 26,0.000000
433,FALPA,0.000000


The results show that the substitution calculation is not always accurate to predict similarity.
For BETON, CBS BETON should be higher than BERGON.

# Difflib
Comparison of Levenshtein and difflib: 
https://stackoverflow.com/questions/6690739/high-performance-fuzzy-string-comparison-in-python-use-levenshtein-or-difflib

In [10]:
s = difflib.SequenceMatcher(None, 'MONOCIBEC', 'FINCIBEC (MONOCIBEC)')

In [11]:
s.ratio()

0.6206896551724138

In [12]:
s.quick_ratio()

0.6206896551724138

In [13]:
s.real_quick_ratio()

0.6206896551724138

In [16]:
def sequence_matcher(compared_brand, series_of_brand):
    
    res_ratio = pd.Series(index=range(len(series_of_brand)), name='difflib_ratio')

    for i in range(len(series_of_brand)):
        s = difflib.SequenceMatcher(None, compared_brand, series_of_brand[i])
        res_ratio[i] = s.ratio()

    res_merged = pd.merge(series_of_brand, res_ratio, left_index=True, right_index=True)
    sorted_ratios = res_merged.sort_values(by='difflib_ratio', ascending=False)

    return sorted_ratios

sequence_matcher('MONOCIBEC', brands_2)

/var/folders/m_/2w2qlxsn13s9qyf11rtgzkkr0000gn/T/ipykernel_81706/27971723.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  res_ratio = pd.Series(index=range(len(series_of_brand)), name='difflib_ratio')


,0,difflib_ratio
459,FINCIBEC (MONOCIBEC),0.620690
330,DISMO FRANCE,0.476190
267,CONCILIO,0.470588
387,EPDM PROVENCE,0.454545
158,BONNEFOY BBCI,0.454545
...,...,...
516,GFD,0.000000
86,AUDY SAS,0.000000
356,DYKA SAS,0.000000
433,FALPA,0.000000


In [ ]:
sequence_matcher()